In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
from keras import backend as K
import time
from IPython.display import clear_output

# Function to load images from a folder
def load_images_from_folder(folder_path, image_size=(128, 128)):
    images = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        img = Image.open(img_path).convert('L')  # Convert to grayscale
        img = img.resize(image_size)  # Resize to (128, 128)
        img_array = np.array(img)
        images.append(img_array)
    images = np.array(images)
    images = np.expand_dims(images, axis=-1)  # Add channel dimension (for grayscale images)
    return images

# Load the image dataset (adjust path based on your setup)
image_folder = r"/content/drive/MyDrive/val"  # Path to the folder containing your images
data = load_images_from_folder(image_folder, image_size=(128, 128))

# Normalize the data to [-1, 1]
data = (data - 127.5) / 127.5

# Constants
NUM_EPOCHS = 10
LATENT_DIM = 100
INPUT_SHAPE = (128, 128, 1)
LEARNING_RATE = 0.0002
MOMENTUM = 0.5
BATCH_SIZE = 64

# Define the Generator (same as before)
def define_generator(latent_dim):
    model = Model()
    # Latent input
    in_lat = layers.Input(shape=(latent_dim,))
    # Foundation for 8x8 image
    n_nodes = 128 * 8 * 8
    gen = layers.Dense(n_nodes)(in_lat)
    gen = layers.LeakyReLU(alpha=0.2)(gen)
    gen = layers.Reshape((8, 8, 128))(gen)

    # Upsample to 16x16
    gen = layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
    gen = layers.LeakyReLU(alpha=0.2)(gen)

    # Upsample to 32x32
    gen = layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
    gen = layers.LeakyReLU(alpha=0.2)(gen)

    # Upsample to 64x64
    gen = layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
    gen = layers.LeakyReLU(alpha=0.2)(gen)

    # Upsample to 128x128
    gen = layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
    gen = layers.LeakyReLU(alpha=0.2)(gen)

    # Output layer - single channel (grayscale) with tanh activation
    out_layer = layers.Conv2D(1, (7,7), activation='tanh', padding='same')(gen)

    # Define model
    model = Model(in_lat, out_layer)
    return model

# Define the Discriminator (same as before)
def define_discriminator(in_shape=(128, 128, 1)):
    model = Model()
    # Image input
    in_image = layers.Input(shape=in_shape)

    # Downsample to 64x64
    fe = layers.Conv2D(64, (4,4), strides=(2,2), padding='same')(in_image)
    fe = layers.LeakyReLU(alpha=0.2)(fe)

    # Downsample to 32x32
    fe = layers.Conv2D(128, (4,4), strides=(2,2), padding='same')(fe)
    fe = layers.LeakyReLU(alpha=0.2)(fe)

    # Downsample to 16x16
    fe = layers.Conv2D(128, (4,4), strides=(2,2), padding='same')(fe)
    fe = layers.LeakyReLU(alpha=0.2)(fe)

    # Downsample to 8x8
    fe = layers.Conv2D(128, (4,4), strides=(2,2), padding='same')(fe)
    fe = layers.LeakyReLU(alpha=0.2)(fe)

    # Flatten feature maps
    fe = layers.Flatten()(fe)

    # Output
    out_layer = layers.Dense(1, activation='sigmoid')(fe)

    # Define model
    model = Model(in_image, out_layer)

    # Compile model
    opt = Adam(learning_rate=0.00001, beta_1=MOMENTUM)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# Define the GAN (same as before)
def define_gan(generator, discriminator):
    # Make discriminator not trainable when training the GAN
    discriminator.trainable = False

    # GAN input (noise)
    gan_input = layers.Input(shape=(LATENT_DIM,))

    # Output of generator
    gen_output = generator(gan_input)

    # Output of discriminator (fake image classification)
    gan_output = discriminator(gen_output)

    # Define GAN model
    gan_model = Model(gan_input, gan_output)

    # Compile GAN
    opt = Adam(learning_rate=LEARNING_RATE, beta_1=MOMENTUM)
    gan_model.compile(loss='binary_crossentropy', optimizer=opt)

    return gan_model

# Function to generate latent points for the generator (same as before)
def generate_latent_points(latent_dim, n_samples):
    x_input = np.random.randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

# Generate fake samples (using the generator) (same as before)
def generate_fake_samples(generator, latent_dim, n_samples):
    latent_points = generate_latent_points(latent_dim, n_samples)
    images = generator.predict(latent_points)
    y = np.zeros((n_samples, 1))
    return images, y

# Train the GAN (same as before)
def train_gan(generator, discriminator, gan_model, dataset, latent_dim, n_epochs=100, n_batch=64):
    batch_per_epoch = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)

    for epoch in range(n_epochs):
        for batch in range(batch_per_epoch):
            # Get real samples
            ix = np.random.randint(0, dataset.shape[0], half_batch)
            X_real, y_real = dataset[ix], np.ones((half_batch, 1))

            # Get fake samples
            X_fake, y_fake = generate_fake_samples(generator, latent_dim, half_batch)

            # Update discriminator with real and fake samples
            # Update discriminator with real and fake samples
            d_loss_real, _ = discriminator.train_on_batch(X_real, y_real)
            d_loss_fake, _ = discriminator.train_on_batch(X_fake, y_fake)

            # Ensure you take only the first value (the loss) if train_on_batch returns a list
            d_loss_real = d_loss_real[0] if isinstance(d_loss_real, list) else d_loss_real
            d_loss_fake = d_loss_fake[0] if isinstance(d_loss_fake, list) else d_loss_fake

            # Prepare points in latent space for the GAN
            X_gan = generate_latent_points(latent_dim, n_batch)
            y_gan = np.ones((n_batch, 1))

            #  Update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            g_loss = g_loss[0] if isinstance(g_loss, list) else g_loss




            # Summarize loss on this batch
            print(f'Epoch: {epoch+1}, Batch: {batch+1}/{batch_per_epoch}, d_loss_real={d_loss_real:.3f}, d_loss_fake={d_loss_fake:.3f}, g_loss={g_loss:.3f}')

        # Save generated images periodically
        if (epoch+1) % 10 == 0:
            save_plot(generator, latent_dim, epoch)

# Function to save generated images (same as before)
def save_plot(generator, latent_dim, epoch, n=10):
    latent_points = generate_latent_points(latent_dim, n*n)
    images = generator.predict(latent_points)

    # Rescale pixel values to [0,1]
    images = (images + 1) / 2.0

    # Plot images
    for i in range(n * n):
        plt.subplot(n, n, 1 + i)
        plt.axis('off')
        plt.imshow(images[i, :, :, 0], cmap='gray')

    # Save plot to file
    filename = f'generated_plot_e{epoch+1}.png'
    plt.savefig(filename)
    plt.close()

# Create the models
discriminator = define_discriminator()
generator = define_generator(LATENT_DIM)
gan_model = define_gan(generator, discriminator)

# Train the GAN
def train_gan(generator, discriminator, gan_model, dataset, latent_dim, n_epochs=100, n_batch=64):
    batch_per_epoch = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)

    for epoch in range(n_epochs):
        for batch in range(batch_per_epoch):
            # Get real samples
            ix = np.random.randint(0, dataset.shape[0], half_batch)
            X_real, y_real = dataset[ix], np.ones((half_batch, 1))

            # Get fake samples
            X_fake, y_fake = generate_fake_samples(generator, latent_dim, half_batch)

            # Update discriminator with real and fake samples
            d_loss_real, _ = discriminator.train_on_batch(X_real, y_real)
            d_loss_fake, _ = discriminator.train_on_batch(X_fake, y_fake)

            # Prepare points in latent space for the GAN
            X_gan = generate_latent_points(latent_dim, n_batch)
            y_gan = np.ones((n_batch, 1))

            # Update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)

            # Summarize loss on this batch
            print(f'Epoch: {epoch+1}, Batch: {batch+1}/{batch_per_epoch}, '
              f'd_loss_real={d_loss_real[0] if isinstance(d_loss_real, list) else d_loss_real:.3f}, '
              f'd_loss_fake={d_loss_fake[0] if isinstance(d_loss_fake, list) else d_loss_fake:.3f}, '
              f'g_loss={g_loss[0] if isinstance(g_loss, list) else g_loss:.3f}')

        # Save generated images periodically
        if (epoch+1) % 10 == 0:
            save_plot(generator, latent_dim, epoch)

# Function to save generated images
def save_plot(generator, latent_dim, epoch, n=10):
    latent_points = generate_latent_points(latent_dim, n*n)
    images = generator.predict(latent_points)

    # Rescale pixel values to [0,1]
    images = (images + 1) / 2.0

    # Plot images
    for i in range(n * n):
        plt.subplot(n, n, 1 + i)
        plt.axis('off')
        plt.imshow(images[i, :, :, 0], cmap='gray')

    # Save plot to file
    filename = f'generated_plot_e{epoch+1}.png'
    plt.savefig(filename)
    plt.close()

# Create the models
discriminator = define_discriminator()
generator = define_generator(LATENT_DIM)
gan_model = define_gan(generator, discriminator)

# Train the GAN
train_gan(generator, discriminator, gan_model, data, LATENT_DIM, NUM_EPOCHS, BATCH_SIZE)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch: 1, Batch: 1/82, d_loss_real=0.715, d_loss_fake=0.704, g_loss=0.704
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


Epoch: 1, Batch: 2/82, d_loss_real=0.707, d_loss_fake=0.703, g_loss=0.703
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Epoch: 1, Batch: 3/82, d_loss_real=0.706, d_loss_fake=0.704, g_loss=0.704
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Epoch: 1, Batch: 4/82, d_loss_real=0.705, d_loss_fake=0.703, g_loss=0.703
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Epoch: 1, Batch: 5/82, d_loss_real=0.705, d_loss_fake=0.703, g_loss=0.703
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Epoch: 1, Batch: 6/82, d_loss_real=0.704, d_loss_fake=0.704, g_loss=0.704
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Epoch: 1, Batch: 7/82, d_loss_real=0.704, d_loss_fake=0.704, g_loss=0.704
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Epoch: 1, Batch: 8/82, d_loss_real=0.704, d_loss_fake=0.704, g_loss=0.704
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Epoch: 1, Batch: 9/82, d_loss_real=0.704, d_loss_fake=0.704, g_loss=0.704
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Epoch: 1, Batch: 10/82, d_loss_real=0.704, d_loss_fake=0.704, g_loss=0.704
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Epoch: 1,

In [ ]:
images = generator.predict(latent_points)

    # Rescale pixel values to [0,1]
images = (images + 1) / 2.0

    # Plot images
for i in range(n * n):
        plt.subplot(n, n, 1 + i)
        plt.axis('off')
        plt.imshow(images[i, :, :, 0], cmap='gray')

NameError: name 'latent_points' is not defined

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

# Function to load images from a folder
def load_images_from_folder(folder_path, image_size=(128, 128)):
    images = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        img = Image.open(img_path).convert('L')  # Convert to grayscale
        img = img.resize(image_size)  # Resize to (128, 128)
        img_array = np.array(img)
        images.append(img_array)
    images = np.array(images)
    images = np.expand_dims(images, axis=-1)  # Add channel dimension (for grayscale images)
    return images

# Load the image dataset (adjust path based on your setup)
image_folder = r"/content/drive/MyDrive/val"  # Path to the folder containing your images
data = load_images_from_folder(image_folder, image_size=(128, 128))

# Normalize the data to [-1, 1]
data = (data - 127.5) / 127.5

# Constants
NUM_EPOCHS = 100
LATENT_DIM = 100
INPUT_SHAPE = (128, 128, 1)
LEARNING_RATE = 0.0002
MOMENTUM = 0.5
BATCH_SIZE = 64

# Detect TPU and initialize
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # Get TPU system
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)  # Strategy to distribute the model on TPU
    print("TPU initialized.")
except ValueError:
    strategy = tf.distribute.get_strategy()  # Default strategy for CPU and single GPU
    print("Using default strategy.")

# Define the Generator
def define_generator(latent_dim):
    model = Model()
    # Latent input
    in_lat = layers.Input(shape=(latent_dim,))
    # Foundation for 8x8 image
    n_nodes = 128 * 8 * 8
    gen = layers.Dense(n_nodes)(in_lat)
    gen = layers.LeakyReLU(alpha=0.2)(gen)
    gen = layers.Reshape((8, 8, 128))(gen)

    # Upsample to 128x128
    for _ in range(4):
        gen = layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same')(gen)
        gen = layers.LeakyReLU(alpha=0.2)(gen)

    # Output layer - single channel (grayscale) with tanh activation
    out_layer = layers.Conv2D(1, (7, 7), activation='tanh', padding='same')(gen)

    # Define model
    model = Model(in_lat, out_layer)
    return model

# Define the Discriminator
def define_discriminator(in_shape=(128, 128, 1)):
    model = Model()
    # Image input
    in_image = layers.Input(shape=in_shape)

    # Downsample to 8x8
    for _ in range(4):
        fe = layers.Conv2D(128, (4, 4), strides=(2, 2), padding='same')(in_image if _ == 0 else fe)
        fe = layers.LeakyReLU(alpha=0.2)(fe)

    # Flatten feature maps
    fe = layers.Flatten()(fe)

    # Output
    out_layer = layers.Dense(1, activation='sigmoid')(fe)

    # Define model
    model = Model(in_image, out_layer)

    # Compile model
    opt = Adam(learning_rate=0.0001, beta_1=MOMENTUM)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# Define the GAN
def define_gan(generator, discriminator):
    # Make discriminator not trainable when training the GAN
    discriminator.trainable = False

    # GAN input (noise)
    gan_input = layers.Input(shape=(LATENT_DIM,))

    # Output of generator
    gen_output = generator(gan_input)

    # Output of discriminator (fake image classification)
    gan_output = discriminator(gen_output)

    # Define GAN model
    gan_model = Model(gan_input, gan_output)

    # Compile GAN
    opt = Adam(learning_rate=LEARNING_RATE, beta_1=MOMENTUM)
    gan_model.compile(loss='binary_crossentropy', optimizer=opt)

    return gan_model

# Function to generate latent points for the generator
def generate_latent_points(latent_dim, n_samples):
    x_input = np.random.randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

# Generate fake samples
def generate_fake_samples(generator, latent_dim, n_samples):
    latent_points = generate_latent_points(latent_dim, n_samples)
    images = generator.predict(latent_points)
    y = np.zeros((n_samples, 1))
    return images, y

# Function to save generated images
def save_plot(generator, latent_dim, epoch, n=10):
    latent_points = generate_latent_points(latent_dim, n*n)
    images = generator.predict(latent_points)

    # Rescale pixel values to [0,1]
    images = (images + 1) / 2.0

    # Plot images
    for i in range(n * n):
        plt.subplot(n, n, 1 + i)
        plt.axis('off')
        plt.imshow(images[i, :, :, 0], cmap='gray')

    # Save plot to file
    filename = f'generated_plot_e{epoch+1}.png'
    plt.savefig(filename)
    plt.close()

# Train the GAN
# Define the GAN training function with checkpoints
def train_gan(generator, discriminator, gan_model, dataset, latent_dim, n_epochs=100, n_batch=64, checkpoint_dir='checkpoints/'):
    batch_per_epoch = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)

    # Create the checkpoint directory if it doesn't exist
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

    # Define a checkpoint callback for the generator
    checkpoint_callback = ModelCheckpoint(
        filepath= '/content/drive/MyDrive/gan_generator_epoch_{epoch:03d}.h5',
        save_weights_only=True,
        save_freq='epoch',
        period=10  # Save every 10 epochs
    )

    for epoch in range(n_epochs):
        for batch in range(batch_per_epoch):
            # Get real samples
            ix = np.random.randint(0, dataset.shape[0], half_batch)
            X_real, y_real = dataset[ix], np.ones((half_batch, 1))

            # Get fake samples
            X_fake, y_fake = generate_fake_samples(generator, latent_dim, half_batch)

            # Update discriminator with real and fake samples
            d_loss_real = discriminator.train_on_batch(X_real, y_real)
            d_loss_fake = discriminator.train_on_batch(X_fake, y_fake)

            # Prepare points in latent space for the GAN
            X_gan = generate_latent_points(latent_dim, n_batch)
            y_gan = np.ones((n_batch, 1))

            # Update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)

            # Summarize loss on this batch
            print(f'Epoch: {epoch+1}, Batch: {batch+1}/{batch_per_epoch}, '
                f'd_loss_real={d_loss_real[0] if isinstance(d_loss_real, list) else d_loss_real:.3f}, '
                f'd_loss_fake={d_loss_fake[0] if isinstance(d_loss_fake, list) else d_loss_fake:.3f}, '
                f'g_loss={g_loss[0] if isinstance(g_loss, list) else g_loss:.3f}')

        # Save generated images periodically
        if (epoch+1) % 10 == 0:
            save_plot(generator, latent_dim, epoch)

        # Save the model weights periodically using the callback
        checkpoint_callback.on_epoch_end(epoch)


# Create the models within the TPU strategy scope
with strategy.scope():
    discriminator = define_discriminator()
    generator = define_generator(LATENT_DIM)
    gan_model = define_gan(generator, discriminator)

# Train the GAN
train_gan(generator, discriminator, gan_model, data, LATENT_DIM, NUM_EPOCHS, BATCH_SIZE)
